# Example Notebook for Checkpoint Plots
A collection of functions to facilitate the comparison of intermediate data products across different reduction and analysss pipelines.

In [1]:
from sys import path
# Add the highest level of the jwst-mtl repo to path
soss_path = '/Users/michaelradica/Documents/GitHub/jwst-mtl/'
path.insert(1, soss_path)

# Import the checkpoint_plot code
import SOSS.checkpoint_plots.checkpoint_plots as checkpoints

Make a plot to assess the quality of the order decontamination performed by ATOCA.

In [3]:
# Path to the SOSS observation fits file. Ideally, whatever was passed to ATOCA via the
# Extract1dStep of the JWST DMS
observation_filepath = 
# Path to the decontaminated SOSS orders constructed by ATOCA. These will generally have
# the suffix '_SossExtractModel.fits'.
# ** Note **: the soss_modelname argument must be specified when calling Extract1dStep to
# produce this files.
soss_model_filepath = 

TypeError: __init__() missing 2 required positional arguments: 'datafile' and 'modelfile'

In [ ]:
# Initialize the DecontaminationPlot
dp = checkpoints.DecontaminationPlot(observation_filepath, soss_model_filepath)

In [ ]:
# Make the plot
# By default a plot is made for the first integration of the TSO. This can be changed by
# passing a list of integrations to the 'integrations' argument. Alternative, pass 'all' to
# plot the deconamination for every integration.
# To output the plots to a pdf file, specify a file name using the 'savefile' argument - useful
# if comparing many integrations.
dp.make_decontamination_plot()